In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet34
import pickle
import numpy as np
import argparse
import os
from PIL import Image
import pandas as pd
import random

print("env is done")

env is done


#### 读取并打包ANIMAL-10N数据集

In [14]:
file_path = 'autodl-tmp/ANIMAL-10N'
# 训练数据
file_path = os.path.join(file_path,'testing')

# 测试数据
# file_path = os.path.join(file_path,'testing')

classesname_animal = {0: "Cat", 1: "Lynx", 2: "Wolf", 3: "Coyote", 4: "Cheetah", 5: "jaguar", 6: "Chimpanzee", 7: "Orangutan", 8: "Hamster", 9: "Guinea pig"}

images_animal = []
labels_animal = []

images_file = os.listdir(file_path)

for image_file in images_file:
    image_path = os.path.join(file_path,image_file)
    label = int(image_file[0])
    image = Image.open(image_path)
    image = image.convert('RGB')
    image = np.array(image)
    images_animal.append(image)
    labels_animal.append(label)

In [15]:
# 数据增强（训练集和测试集）
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 自定义 Dataset 类
class AnimalDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # 将图像转换为 PIL 格式
        image = Image.fromarray(np.array(self.images[idx]))  # 确保图像是 PIL 格式
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        if self.transform:
            image = self.transform(image)
            
        return image, label
    
length = len(images_animal)
# 将 20% 的数据作为训练集，剩余 80% 的数据作为测试集
# 设置随机种子以确保结果一致
random.seed(42)

# 将图像和标签组合起来进行打乱
data = list(zip(images_animal, labels_animal))
random.shuffle(data)

# 解压打乱后的数据
shuffled_images, shuffled_labels = zip(*data)

# 获取数据集的长度
length = len(shuffled_images)

# 划分训练集（20%）和测试集（80%）
trainset_animal = AnimalDataset(
    images=shuffled_images[:int(0.6 * length)], 
    labels=shuffled_labels[:int(0.6 * length)],
    transform=transforms_train
)
testset_animal = AnimalDataset(
    images=shuffled_images[int(0.6 * length):], 
    labels=shuffled_labels[int(0.6 * length):],
    transform=transforms_test
)

In [16]:
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 256

train_loader = DataLoader(trainset_animal, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(testset_animal, batch_size=batch_size, shuffle=False, num_workers=4)

In [17]:
# 定义 ResNet 模型
def initialize_model(weights_path):
    model = resnet34(weights=None)
    model.fc = nn.Linear(model.fc.in_features, 10)
    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict, strict=False)
    return model.to(device)

# 定义损失函数和优化器
def initialize_optimizer(model):
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
    return optimizer, scheduler

# 训练函数
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets).mean()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    
    avg_loss = running_loss / len(train_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy

# 测试函数
def test(model, test_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets).mean()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    avg_loss = running_loss / len(test_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy

# 创建文件夹
os.makedirs('log', exist_ok=True)
os.makedirs('weight', exist_ok=True)

In [23]:
print(f"-----------------直接训练开始，noise标签为 -----------------")
model = initialize_model('weight/resnet34_animal_raw_noisy_label.pth')
optimizer, scheduler = initialize_optimizer(model)

criterion = nn.CrossEntropyLoss(reduction='none')

train_losses, train_accuracies, test_losses, test_accuracies = [], [], [], []

epochs = 100
# 直接训练
for epoch in range(epochs):
    train_loss, train_accuracy = train(model, train_loader, criterion, optimizer)
    test_loss, test_accuracy = test(model, test_loader, criterion)
    scheduler.step()
        
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    print(f'Epoch {epoch + 1} - Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
            f'Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.2f}%')
    if train_accuracy > 99.999:
        break

# 保存训练日志
pd.DataFrame({
    'Epoch': range(1, len(train_losses) + 1),
    'Train Loss': train_losses,
    'Train Accuracy': train_accuracies,
    'Test Loss': test_losses,
    'Test Accuracy': test_accuracies
}).to_csv(f'log//resnet34_animal_raw_finetuning_all', index=False)
    
torch.save(model.state_dict(), f'weight//resnet34_animal_raw_finetuning_all.pth')
print(f"训练过程已保存到 'log//resnet34_animal_raw_finetuning_all.csv'，模型权重保存为 'weight/resnet34_animal_raw_finetuning_all.pth'")

-----------------直接训练开始，noise标签为 -----------------
Epoch 1 - Train Loss: 1.3602, Train Acc: 53.07%, Test Loss: 113.3831, Test Acc: 8.25%
Epoch 2 - Train Loss: 1.4738, Train Acc: 48.10%, Test Loss: 6.9003, Test Acc: 17.85%
Epoch 3 - Train Loss: 1.4425, Train Acc: 48.93%, Test Loss: 1.6779, Test Acc: 40.20%
Epoch 4 - Train Loss: 1.3909, Train Acc: 50.83%, Test Loss: 1.6139, Test Acc: 41.15%
Epoch 5 - Train Loss: 1.3879, Train Acc: 50.50%, Test Loss: 1.5007, Test Acc: 48.85%
Epoch 6 - Train Loss: 1.3706, Train Acc: 51.73%, Test Loss: 1.5037, Test Acc: 50.05%
Epoch 7 - Train Loss: 1.3579, Train Acc: 51.30%, Test Loss: 1.4559, Test Acc: 50.35%
Epoch 8 - Train Loss: 1.3447, Train Acc: 52.93%, Test Loss: 1.5875, Test Acc: 44.25%
Epoch 9 - Train Loss: 1.3653, Train Acc: 52.33%, Test Loss: 1.4823, Test Acc: 47.15%
Epoch 10 - Train Loss: 1.3276, Train Acc: 53.13%, Test Loss: 1.5754, Test Acc: 45.00%
Epoch 11 - Train Loss: 1.3072, Train Acc: 54.30%, Test Loss: 1.3797, Test Acc: 53.25%
Epoch 12 - 

In [25]:
print(f"-----------------输出层fine-tuning -----------------")
model = initialize_model('weight/resnet34_animal_raw_noisy_label.pth')

# 冻结所有参数（不计算梯度）
for param in model.parameters():
    param.requires_grad = False

# 仅对最后一层（model.fc）的参数解冻
for param in model.fc.parameters():
    param.requires_grad = True

# 仅将最后一层的参数传递给优化器
optimizer, scheduler = initialize_optimizer(model)

lr = 0.1
optimizer = optim.SGD(model.fc.parameters(), lr=lr, momentum=0.9)

criterion = nn.CrossEntropyLoss(reduction='none')

train_losses, train_accuracies, test_losses, test_accuracies = [], [], [], []

epochs = 100
# 直接训练
for epoch in range(epochs):
    train_loss, train_accuracy = train(model, train_loader, criterion, optimizer)
    test_loss, test_accuracy = test(model, test_loader, criterion)
    scheduler.step()
        
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    print(f'Epoch {epoch + 1} - Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
            f'Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.2f}%')
    if train_accuracy > 99.999:
        break

# 保存训练日志
pd.DataFrame({
    'Epoch': range(1, len(train_losses) + 1),
    'Train Loss': train_losses,
    'Train Accuracy': train_accuracies,
    'Test Loss': test_losses,
    'Test Accuracy': test_accuracies
}).to_csv(f'log//resnet34_animal_raw_finetuning_{lr}fc.csv', index=False)
    
torch.save(model.state_dict(), f'weight//resnet34_animal_raw_finetuning_{lr}fc.pth')
print(f"训练过程已保存到 'log//resnet34_animal_raw_finetuning_{lr}fc.csv'，模型权重保存为 'weight/resnet34_animal_raw_finetuning_{lr}fc.pth'")

-----------------输出层fine-tuning -----------------
Epoch 1 - Train Loss: 1.0341, Train Acc: 64.53%, Test Loss: 1.3811, Test Acc: 63.45%
Epoch 2 - Train Loss: 1.0215, Train Acc: 64.03%, Test Loss: 1.2596, Test Acc: 67.40%
Epoch 3 - Train Loss: 1.0358, Train Acc: 63.70%, Test Loss: 1.3040, Test Acc: 65.15%
Epoch 4 - Train Loss: 1.0220, Train Acc: 63.87%, Test Loss: 1.3896, Test Acc: 63.55%
Epoch 5 - Train Loss: 0.9994, Train Acc: 64.43%, Test Loss: 1.2732, Test Acc: 65.95%
Epoch 6 - Train Loss: 1.0465, Train Acc: 62.83%, Test Loss: 1.2744, Test Acc: 65.05%
Epoch 7 - Train Loss: 0.9966, Train Acc: 65.30%, Test Loss: 1.2900, Test Acc: 65.65%
Epoch 8 - Train Loss: 1.0352, Train Acc: 64.83%, Test Loss: 1.2886, Test Acc: 65.60%
Epoch 9 - Train Loss: 0.9854, Train Acc: 64.87%, Test Loss: 1.2068, Test Acc: 68.40%
Epoch 10 - Train Loss: 1.0343, Train Acc: 63.80%, Test Loss: 1.3395, Test Acc: 64.15%
Epoch 11 - Train Loss: 1.0166, Train Acc: 64.47%, Test Loss: 1.3298, Test Acc: 65.40%
Epoch 12 - Tr